# Optimizer: 3X LYF Delta Neutral with Rebalancing

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ClaudeF4491/defi_leveraged_yield_farming_sim/blob/main/optimize_rebalance_threshold.ipynb)

**Overview**

This is very similar to `farming_simulator.ipynb`, but rather than plotting results for a single rebalancing threshold, it grid-searches several thresholds and plots the performance. That way one can assess which is the best rebalance threshold for a given scenario.

**Setup**
- See `farming_simulator.ipynb`. Same setup


# Google Colab Setup
---


In [ ]:
# Check if on Colab
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

# Download packages if on Colab
if IN_COLAB:
    # Download source code and add to PYTHONPATH if doesn't exist
    import sys
    import os
    project_name = "defi-farming-sim"
    if not os.path.isdir(project_name):
      # Fetch source modules
      !git clone https://github.com/ClaudeF4491/{project_name}.git
      !cp {project_name}/requirements.txt ./
      sys.path.append(f"./{project_name}")

      # Move the data to this dir
      !mv -f ./{project_name}/data ./

# Extract
!tar xf ./data/dataset_20220624.tar.gz -C ./data/

# Install Packages
---

In [ ]:
!pip install -r requirements.txt

# Imports
---

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from tqdm import tqdm

# Import all the support strategies & scenarios so a user of this notebook can select any of them.
import scenarios
import simulation
import strategies.rewards as reward_strategies
import strategies.trading as trading_strategies

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

<a id='configuration_setup'></a>
# Configuration & Scenario Setup
---

## Parameters

In [ ]:
"""
Optimizer Configuration
"""
search_vals = np.arange(0, 1.0, 0.01)

In [ ]:
"""
Configuration that is always used, regardless of scenario
"""
# Initial amount invested
initial_cap = 10000

# Swap fee, note: 0.01 = 1%
fee_swap = 0.003

# Cost of gas for each transaction, in dollars
fee_gas = 1.0

# Open a leveraged position
# Options: [1.0, 2.0, >2.0]
leverage = 3

# Price of reward token
reward_token_price = 1.0

# Liquidation threshold
# Note: Francium threshold is 83.3%. See Francium reference at top.
# Currently unused. For plotting purposes only.
liquidation_threshold = 0.833

"""
Configuration that is used if the scenario does not internally generate the data itself
"""
# Earnings APYs, note: 1.0 = 100%
apy_trading_fee = 0.2
apy_reward = 0.1

# Borrow / Lending APYs
apy_borrow_token0 = 0.12
apy_borrow_token1 = 0.08

# Start date
# Optional start date offset that's used if not None and if not overwritten by trading strategy
start_offset = None
start_date = None


In [ ]:
"""
Configuration that is used if `create_history_from_files()` is selected as the scenario. 
That generator derives all pool APYs, borrow APYs, and prices
"""
# Token names, according to CREAM, to lookup borrow rates. 
# May differ from CoinDix pool symbol name
cream_token0_name = "USDC.e" 
cream_token1_name = "WAVAX"

# CoinGecko config
token1_coingecko_id = "avalanche-2"
reward_token_coingecko_id= "joe"

# CoinDix parameters
coindix_pair = "USDC.e-AVAX"
protocol = "trader joe"
chain = "avalanche"

<a id='reward_strategy_definition'></a>
## Reward Strategy Definition
Define a reward function using one found in [Reward Token Strategies](#reward_token_strategies) section. Or create your own and reference it here.

In [ ]:
# Define parameters for what to do with rewards
# To HODL, use any SellRewardsStrategy(sell_amount=0)
rewards_sell_amount = 1.0

# Options: SellRewardsStrategy, CompoundRewardsStrategy
rewards_cls = reward_strategies.SellRewardsStrategy(sell_amount=rewards_sell_amount)
# rewards_cls = reward_strategies.CompoundRewardsStrategy(sell_amount=rewards_sell_amount)

<a id='price_movement_scenario'></a>
## Price Movement Scenario

Uncomment the scenario to try it. Or create your own from a function in the [Scenario Generators](#scenario_generators) section.

In [ ]:
"""
Uncomment to create a small 4-day example that unit-tests: no change, very negative price change, very positive price change
"""
# pp = scenarios.create_small_example()


"""
Uncomment this to run for one year without any price change -- useful to test APY growths and fees without price impact
"""
# pp = scenarios.create_no_price_change_example(365)


"""
Uncomment this to run for one year, decaying price change to 80% losses -- useful to see dramatic effects on investment
"""
# pp = scenarios.create_linear_price_change_example(365, token0_price=1.0, token1_price_initial=0.1, token1_price_final=0.02)


"""
Uncomment this to run for one year, increasing price 5X -- useful to see dramatic effects on investment
"""
# pp = scenarios.create_linear_price_change_example(365, token0_price=1.0, token1_price_initial=0.1, token1_price_final=0.5)


"""
Uncomment this to run for one year, without any non-safe-asset (NSA) price change, but the reward token drains to near zero.
This is an example of typical farming token where it may be better to sell every day instead of compound or hold, 
depending on initial capital and gas fees.
"""
# pp = scenarios.create_linear_price_change_example(
#     365, token0_price=1.0, token1_price_initial=0.1, token1_price_final=0.1, reward_token_price_initial=1.0, reward_token_price_final=0.01
# )

"""
Uncomment this to go from price of 1.0 down to 0.2 and back to 0.1 within a year. Tests to see the effect of returning to no IL 
when price dumps and comes back
"""
# pp = scenarios.create_linear_and_back_example(365, token0_price=1.0, token1_price_base=0.1, token1_price_peak=0.01)

"""
Uncomment this to go from price of 1.0 down to 5.0 (5X) and back to 0.1 within a year. Tests to see the effect of returning to 
no IL when price pumps and comes back
"""
# pp = scenarios.create_linear_and_back_example(365, token0_price=1.0, token1_price_base=0.1, token1_price_peak=0.3)

"""
Uncomment this to take a random walk. 
Tips: 
- Set seed to None for random every time, or integer to be deterministic
- Set bias and variance based in terms of daily-percent movement. 
- See defaults for reasonable starts. Try plotting first as well.
- Set bias to positive to walk upward, negative to walk downward, or 0 to randomly move stationary. 
- Set variance to define how much the walk should move each step.
"""
# pp = scenarios.create_random_walk_example(365, token1_price_initial=10, bias=-0.002, variance=0.005, seed=1234)

"""
Uncomment this to run a real-world example using actual historical data.
Reads in AVAX-USDC.e pool example from CoinDix/CREAM and prices from coingecko
Assumes 10% of total APY is rewards (coindix doesn't report reward APY for this pool, so just an assumption)
"""
pp = scenarios.create_history_from_files(
    coindix_pair,
    protocol,
    chain,
    cream_token0_name, 
    cream_token1_name,
    token1_coingecko_id,
    reward_token_coingecko_id,
    reward_apy_ratio=0.10
)

# Suppress auto-comment print above
print("")

<a id='trade_strategy_definition'></a>
## Trade Strategy Definition
Initialize a trade strategy object found in [Trade Strategies](#trade_strategies)

In [ ]:
# Set start date based on desired input, if not already assigned
if start_date is None:
    start_date = pp.index[0]
    if start_offset is not None:
        start_date = pp.index[0] + pd.Timedelta(start_offset, unit="days")
print(f"Start Date: {start_date}")

# Initialize
---

In [ ]:
# Set start date based on desired input, if not already assigned
if start_date is None and start_offset is not None:
    start_date = pp.index[0] + pd.Timedelta(start_offset, unit="days")
print(f"Start Date: {start_date}")

# Run Simulations
---

In [ ]:
print(f"Found {len(search_vals)} search candidates.")

In [ ]:
results = list()
open_on_start = True
for sv in tqdm(search_vals):
    strategy_cls = trading_strategies.RebalanceStrategy(
        pp["token1_price"][0], 
        sv,
        leverage,
        1.0,
        fee_swap=fee_swap/10.0,  # In real-world, the swap will be a balancing of a small percentage of the total position
        fee_gas=fee_gas/3.0
    )
    
    zz = pp.copy()
    zz = simulation.simulate(
        zz,
        strategy_cls,
        rewards_cls,
        initial_cap,
        leverage,
        start_date,
        fee_gas,
        fee_swap,
        reward_token_price,
        apy_trading_fee,
        apy_reward,
        apy_borrow_token0,
        apy_borrow_token1,
        open_on_start
    )
    
    rr = simulation.summarize_results(zz)
    rr["search_config"] = sv
    results.append(rr)

# Evaluate
---

In [ ]:
dfr = pd.DataFrame(results)

In [ ]:
dfr.plot.scatter(x="search_config", y="num_trade_strategy_executions", grid="on", figsize=(12, 8))

In [ ]:
dfr.plot(x="search_config", y="final_annualized_apr", grid="on", figsize=(12, 8))